In [3]:
import os
import operator
import pickle

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from typing import List, Tuple, Dict
from snn.layers import SCTNLayer
from snn.spiking_network import SpikingNetwork
from snn.spiking_neuron import create_SCTN, BINARY
from scripts.rwcp_resonators import create_neuron_for_labeling
from utils import neurons_labels, save_network_weights, load_network_weights
from utils.save_model import save_model


In [22]:
def create_neuron_for_labeling(synapses_weights):
    neuron = create_SCTN()
    neuron.synapses_weights = synapses_weights
    neuron.leakage_factor = np.random.randint(4) + 1
    neuron.leakage_period = (np.random.randint(4) + 1) * 100
    neuron.theta = -1e-4 / neuron.leakage_factor
    neuron.threshold_pulse = 50 * len(synapses_weights)
    neuron.activation_function = BINARY
    return neuron

def create_random_network(freqs, clk_freq, n_neurons):
    # create network with n neurons as labels.
    # The neurons are not learning yet.
    network = SpikingNetwork(clk_freq)
    labels_neurons = [
        create_neuron_for_labeling(np.random.random(len(freqs)) * 5 + 5)
        for _ in range(n_neurons)
    ]
    network.add_layer(SCTNLayer(labels_neurons))
    for neuron in network.layers_neurons[-1].neurons:
        network.log_out_spikes(neuron._id)
    return network


def train_test_files(label, train_ratio=.5, seed=42):
    files_names = np.array(os.listdir(f"../datasets/RWCP_spikes/{label}"))

    np.random.seed(seed)
    shuffle = np.random.permutation(len(files_names))
    files_names = files_names[shuffle]
    train = files_names[:int(len(files_names) * train_ratio)]
    test = files_names[int(len(files_names) * train_ratio):]
    return train, test


def get_signals(test: bool, labels, seed=42, train_ratio=.5, oversample=False) -> List[Tuple[str, str]]:

    files_names = {
        label: train_test_files(label, seed=seed, train_ratio=train_ratio)[1 if test else 0]
        for label in labels
    }

    np.random.seed(seed)
    if oversample:
        max_samples = max(map(len, files_names.values()))

        def oversample(files):
            extra_samples = max_samples - len(files)
            choices = np.random.choice(len(files), extra_samples)
            return np.concatenate([files, files[choices]])

        files_names = {
            label: oversample(files_names[label])
            for label in labels
        }

    signals_files = [
        f'{label}/{f}'
        for label, files in files_names.items()
        for f in files
    ]
    labels = [
        label
        for  label, files in files_names.items()
        for _ in range(len(files))
    ]

    res = np.array(list(zip(signals_files, labels)))

    shuffle = np.random.permutation(len(res))
    return res[shuffle]


def activate_stdp_to_same_label_neurons(network: SpikingNetwork,
                                        label: str,
                                        ltp,
                                        ltd):
    time_to_learn = 15e-3
    tau = network.clk_freq * time_to_learn / 2

    for neuron in network.layers_neurons[-1].neurons:
        if neuron.label == label:
            neuron.set_stdp(ltp, ltd, tau, clk_freq, 30, -20)


def activate_stdp_to_different_label_neurons(network: SpikingNetwork,
                                             label: str,
                                             ltp,
                                             ltd):
    time_to_learn = 15e-3
    tau = network.clk_freq * time_to_learn / 2

    for neuron in network.layers_neurons[-1].neurons:
        if neuron.label != label and neuron.label is not None:
            neuron.set_stdp(ltp, ltd, tau, clk_freq, 30, 0)


def load_spikes_data(file_name, freqs, length=None):
    spikes = pd.DataFrame \
        .from_dict(dict(
            np.load(f'..\datasets\RWCP_spikes\\{file_name}')
        ))
    columns = [f'f{f}' for f in freqs]
    res = spikes[columns].to_numpy()
    if length is None:
        return res
    last_start_point = len(res) - length
    if last_start_point <= 0:
        return res

    start_point = np.random.randint(last_start_point)
    return res[start_point:start_point+length]


def tag_neuron_a_label(network: SpikingNetwork,
                       post_spikes: np.ndarray,
                       label: str,
                       neurons_labels_counter: Dict[int, Dict[str, int]]):
    post_spikes = post_spikes.copy()
    arg_most_active_neuron = np.argmax(post_spikes)
    if post_spikes[arg_most_active_neuron] == 0:
        return False
    most_active_neuron = network.layers_neurons[-1].neurons[arg_most_active_neuron]
    neuron_labels_counter = neurons_labels_counter[most_active_neuron._id]
    # print(f'{most_active_neuron._id}: {neuron_labels_counter} + {label}')
    neuron_labels_counter[label] += 1
    max_label = max(neuron_labels_counter, key=neuron_labels_counter.get)
    ratio_of_max_label = neuron_labels_counter[max_label] / sum(neuron_labels_counter.values())
    if most_active_neuron.label != max_label:
        most_active_neuron.label = max_label
        return True
    return False
    # count_labels = sum(neuron_labels_counter.values())
    # print(f'{most_active_neuron._id} for {label}. with {ratio_of_max_label}')
    # # print(f'{most_active_neuron._id} for {label}. count_labels: {count_labels} with {ratio_of_max_label}')
    # if count_labels >= 3 and ratio_of_max_label >= 1/3:
    # # if ratio_of_max_label >= 1/2:
    #     if most_active_neuron.label != max_label:
    #         most_active_neuron.label = max_label
    #         return True
    # else:
    #     most_active_neuron.label = None
    #
    # return False

def amplify_inactive_neurons(
        network: SpikingNetwork,
        label: str,
):
    for i, neuron in enumerate(network.layers_neurons[-1].neurons):
        if neuron.label == label:
            neuron.synapses_weights *= 1.025


def learning_process(
        network: SpikingNetwork,
        train_signals: List[Tuple[str, str]],
        neurons_encoder,
        epochs: int,
        singal_length
):
    labels = list(neurons_encoder.keys())
    neurons_encoder[None] = '-'

    neurons_label_counter = {
        neuron._id: {
            label: 0
            for label in labels
        }
        for neuron in network.neurons
    }

    neurons_removed = 0
    total_runs = epochs * len(train_signals)

    leakage_factor_arr = np.array([neuron.leakage_factor for neuron in network.neurons])
    with tqdm(total=total_runs) as pbar:
        for epoch in range(epochs):
            permutation_audio_file_indices = np.random.permutation(len(train_signals))

            sum_epoch_accuracy_sum = 0
            max_epoch_accuracy_sum = 0
            mean_epoch_accuracy_sum = 0
            sum_epoch_accuracy_count = 0
            max_epoch_accuracy_count = 0
            mean_epoch_accuracy_count = 0

            for i, (signal, label) in enumerate(train_signals[permutation_audio_file_indices]):
                stdp_lt_decay = .965 ** epoch
                activate_stdp_to_same_label_neurons(network, label,
                                                    ltp=1e-3 * stdp_lt_decay,
                                                    ltd=-8e-4 * stdp_lt_decay)
                activate_stdp_to_different_label_neurons(network, label,
                                                         ltp=-8e-5 * stdp_lt_decay,
                                                         ltd=0)

                spikes = load_spikes_data(signal, freqs, length=singal_length)
                post_spikes = network.input_full_data_spikes(
                    spikes,
                    False       # don't stop_on_first_spike
                )

                # normalize iot with leakage factor

                post_spikes = post_spikes / leakage_factor_arr
                if tag_neuron_a_label(network, post_spikes, label, neurons_label_counter):
                    # if new neuron got a label,
                    if epoch < 0:#epochs/2:
                        # ill add another new unlabeled neuron to learn only if it has enough samples
                        # to learn.
                        neuron = create_neuron_for_labeling(np.random.random(len(freqs)) * 10)
                        network.add_neuron(neuron, 0)

                        neurons_label_counter[neuron._id] = {
                            label: 0
                            for label in labels
                        }
                amplify_inactive_neurons(network, label)

                # prepare for new input
                network.reset_learning()
                network.reset_input()

                predicted_sum, prediction_spikes_sum = predict_label_by_sum(
                    neurons_encoder.keys(),
                    network.layers_neurons[-1].neurons,
                    post_spikes
                )

                predicted_max, prediction_spikes_max = predict_label_by_max(
                    network.layers_neurons[-1].neurons,
                    post_spikes
                )


                predicted_mean, prediction_spikes_mean = predict_label_by_mean(
                    neurons_encoder.keys(),
                    network.layers_neurons[-1].neurons,
                    post_spikes
                )

                sum_epoch_accuracy_count += 1
                sum_epoch_accuracy_sum += label == predicted_sum
                max_epoch_accuracy_count += 1
                max_epoch_accuracy_sum += label == predicted_max
                mean_epoch_accuracy_count += 1
                mean_epoch_accuracy_sum += label == predicted_mean
                pbar.set_description(
                    f"{sum_epoch_accuracy_sum/sum_epoch_accuracy_count * 100:.1f}%|"
                    f"{max_epoch_accuracy_sum/max_epoch_accuracy_count * 100:.1f}%|"
                    f"{mean_epoch_accuracy_sum/mean_epoch_accuracy_count * 100:.1f}%|"
                    f" {label} {'V' if label == predicted_sum or label == predicted_max else 'X'} {prediction_spikes_max}/{prediction_spikes_sum}  | n removed: {neurons_removed} | {neurons_labels(network.layers_neurons[-1].neurons[:len(post_spikes)], encoder=neurons_encoder, spikes=post_spikes)}")
                pbar.update()

                # if there are neurons with similar weights or zero weights, remove them.
                if i > 0 and i % 25 == 0:
                    neurons_removed += network.remove_irrelevant_neurons(1) # threshold
                    leakage_factor_arr = np.array([neuron.leakage_factor for neuron in network.neurons])
                    amplify_inactive_neurons(network, None)

            print(
                f"{sum_epoch_accuracy_sum/sum_epoch_accuracy_count * 100:.1f}%|"
                f"{max_epoch_accuracy_sum/max_epoch_accuracy_count * 100:.1f}%|"
                    f"{mean_epoch_accuracy_sum/mean_epoch_accuracy_count * 100:.1f}%|"
                f" Total neurons {len(network.layers_neurons[-1].neurons)} | total removed {neurons_removed}"
            )

            if epoch == 10:
                # fishy way to remove unlabeld neurons
                for neuron in network.neurons:
                    if neuron.label is None:
                        neuron.synapses_weights = np.zeros(shape=neuron.synapses_weights)
                neurons_removed += network.remove_irrelevant_neurons(1) # threshold
                leakage_factor_arr = np.array([neuron.leakage_factor for neuron in network.neurons])


            if epoch % 5 == 0:
                save_model(network, path=f'networks/semi_supervised_learning_e{epoch}.pickle')

def predict_label_by_sum(labels, neurons, post_spikes):
    labels_counter = {label: 0 for label in labels}
    for i, neuron in enumerate(neurons):
        if neuron.label is not None:
            labels_counter[neuron.label] += post_spikes[i]
    return max(labels_counter.items(), key=operator.itemgetter(1))[0], \
        dict(sorted(labels_counter.items(), key=lambda item: item[1], reverse=True))


def predict_label_by_max(neurons, post_spikes):
    arg_most_active_neuron = np.argmax(post_spikes)
    most_active_neuron = neurons[arg_most_active_neuron]
    return most_active_neuron.label, post_spikes[arg_most_active_neuron]


def predict_label_by_mean(labels, neurons, post_spikes):
    labels_counter = {label: [0, 0] for label in labels}
    for i, neuron in enumerate(neurons):
        if neuron.label is not None:
            labels_counter[neuron.label][0] += post_spikes[i]
            labels_counter[neuron.label][1] += 1

    labels_mean = {
        label: spikes_sum/neurons_count if neurons_count > 0 else 0
        for label, [spikes_sum, neurons_count] in labels_counter.items()
    }
    return max(labels_mean.items(), key=operator.itemgetter(1))[0], \
        dict(sorted(labels_mean.items(), key=lambda item: item[1], reverse=True))


def semi_supervised_learning(
        freqs,
        clk_freq,
        unlabeled_neurons,
        signal_length=None,
        epochs=1,
        train_ratio=.5):
    neurons_encoder = {
        # 'bottle2': '🍶',
        'dice1': '🎲',
        'cherry1': '🌳',
        'metal05': '🧇',
    }
    network = create_random_network(freqs, clk_freq, unlabeled_neurons)
    train_signals_files = get_signals(test=False,
                                      labels=neurons_encoder.keys(),
                                      train_ratio=train_ratio,
                                      oversample=True)
    learning_process(
        network,
        train_signals_files,
        neurons_encoder=neurons_encoder,
        epochs=epochs,
        singal_length=signal_length
    )
    save_model(network, path=f'networks/semi_supervised_learning.pickle')
    return network

In [23]:
clk_freq = int(1.536 * (10 ** 6) * 2)

freqs = [
    # 236, 751, 887, 1046, 1235, 2029, 2825, 3934, 5478
    200, 236, 278, 328, 387, 457,
    751, 887, 1046, 1235, 1457,
    1719, 2029, 2825, 3934, 5478
]
network = semi_supervised_learning(freqs,
                                   clk_freq,
                                   unlabeled_neurons=50,
                                   # signal_length=clk_freq // 5,
                                   epochs=40,
                                   train_ratio=.5)

  0%|          | 0/6000 [00:00<?, ?it/s]

56.7%|57.3%|48.0%| Total neurons 44 | total removed 6
51.3%|52.0%|43.3%| Total neurons 43 | total removed 7
53.3%|52.0%|44.7%| Total neurons 42 | total removed 8
49.3%|48.0%|38.7%| Total neurons 42 | total removed 8
48.0%|50.0%|46.0%| Total neurons 42 | total removed 8
46.7%|48.7%|45.3%| Total neurons 42 | total removed 8
46.7%|51.3%|46.0%| Total neurons 39 | total removed 11
48.7%|51.3%|46.0%| Total neurons 38 | total removed 12
44.7%|46.7%|43.3%| Total neurons 38 | total removed 12
42.0%|44.0%|44.0%| Total neurons 38 | total removed 12
42.0%|45.3%|44.0%| Total neurons 37 | total removed 13
38.0%|37.3%|36.7%| Total neurons 36 | total removed 14
41.3%|46.0%|42.7%| Total neurons 33 | total removed 17
44.7%|45.3%|40.0%| Total neurons 33 | total removed 17
40.7%|43.3%|42.0%| Total neurons 32 | total removed 18
44.0%|44.7%|40.7%| Total neurons 32 | total removed 18
43.3%|47.3%|40.0%| Total neurons 32 | total removed 18
36.0%|43.3%|36.7%| Total neurons 32 | total removed 18
38.7%|38.7%|36.7

In [14]:
clk_freq //20

153600

In [8]:
n = network.layers_neurons[-1].neurons[-2]
n.synapses_weights

array([ 0.11086807, -0.80661419,  1.55629568, -2.5134114 ,  2.43022647,
        3.1839897 , -5.07300176, -0.34431008,  2.78732933])

In [11]:
for n in network.layers_neurons[-1].neurons:
    # n = network.layers_neurons[-1].neurons[-2]
    print(f'm: {np.mean(n.synapses_weights)}, s: {np.sum(n.synapses_weights)}')

m: 0.17418922701791142, s: 1.567703043161203
m: 0.014220146044304747, s: 0.12798131439874272
m: 0.09121399036537664, s: 0.8209259132883897
m: 23.857278257783364, s: 214.7155043200503
m: 3.216422223713165, s: 28.947800013418483
m: 2.9157538741906257, s: 26.24178486771563
m: 14.36104289820265, s: 129.24938608382385
m: 0.11426193986906845, s: 1.028357458821616
m: 0.1479302039393225, s: 1.3313718354539024
m: 0.11139330782818216, s: 1.0025397704536394


In [13]:
np.array([n.synapses_weights for n in network.layers_neurons[-1].neurons])

array([[ 0.42634979, -0.9356769 ,  1.15019126, -2.01583432,  2.13429621,
         3.19053626, -3.04678838, -1.93266906,  2.59729819],
       [ 1.42604832, -1.43266243,  3.18843389,  1.38256309, -5.61179337,
         2.89713318, -2.45024665, -2.66220082,  3.39070609],
       [ 0.36689498,  0.13862808,  0.86473384, -1.25849823,  1.81880796,
         2.95287966, -1.79905318, -4.44386731,  2.18040013],
       [24.98881504, 24.9888165 , 24.99850451, 24.92681483, 24.94278241,
        24.98002726, 14.90951792, 24.98330274, 24.99692312],
       [ 1.20164648,  3.41879667,  0.91799066,  0.94156988,  3.11413309,
         9.79510529,  1.7533027 ,  0.17161102,  7.63364423],
       [ 2.88630553,  5.81238221,  1.54362715,  4.81140102,  5.32589433,
         0.51823537,  3.36604278,  1.34414677,  0.6337497 ],
       [14.16051573, 13.29360264, 14.61968991, 12.7890441 , 15.00243343,
        15.38766173, 15.07142736, 13.97815128, 14.9468599 ],
       [ 0.4811429 ,  0.10211204,  1.59119002, -3.48779611,  2

In [ ]:
def test_process(network, test_signal_files):
    labels = [f'{n.label}_{n._id}' for n in network.layers_neurons[-1].neurons]
    predict_results = []
    for signal_file, label in tqdm(test_signal_files):
        spikes = load_spikes_data(signal_file, freqs)
        post_spikes = network.input_full_data_spikes(spikes)
        res = dict(zip(labels, post_spikes))
        res['label'] = label
        predict_results.append(res)
        network.reset_input()

    df = pd.DataFrame.from_records(predict_results)
    df.to_csv('output_spikes/semi_supervised_test.csv', index=False)
    return df


def test_network(network=None,
                 freqs=None,
                 clk_freq=None,
                 n_neurons=None,
                 train_ratio=.5):
    if network is None:
        network = create_random_network(freqs, clk_freq, n_neurons)
        load_network_weights(network, path='neurons_weights/semi_supervised_learning.pickle')
    test_signals_files = get_signals(test=True, train_ratio=train_ratio, oversample=False)
    return test_process(network, test_signals_files)

In [11]:
with open('neurons_weights/semi_supervised_learning_e1.pickle', 'rb') as handle:
    (weights, labels) = pickle.load(handle)

weights = np.array(list(weights.values()))
weights

array([[ 4.35672899e+00,  7.30039317e+00,  4.77161277e-01,
         5.66037210e+00,  1.58646448e+00,  1.20164648e+00,
         3.41879667e+00,  9.17990658e-01,  9.41569883e-01],
       [-1.26426331e+00, -3.72111760e+00,  1.18103599e+00,
         1.64843097e+00,  3.70811759e+00, -2.46478648e+00,
        -1.52717352e+00,  9.16547228e-02,  2.55114505e+00],
       [ 1.87490623e+00, -1.20435093e+01,  5.01675772e-01,
        -2.25845343e+00,  3.64779624e+00,  3.56363266e+00,
        -1.55391389e+01,  2.44251172e+01, -4.00651120e+00],
       [ 3.64591807e-03, -2.23123428e+00,  3.15930862e+00,
        -2.73905917e-02, -8.15936886e-01, -3.59395366e-01,
         1.97396357e-01, -7.46947894e-01,  1.41968974e+00],
       [ 7.32743461e-01, -9.25002166e-02,  3.04672375e+00,
        -1.09355767e+00,  1.19112157e+00,  3.02269213e+00,
         5.90710433e-01, -4.85874360e+00, -2.29784524e+00],
       [-4.62760390e+00, -1.70754592e+00,  1.27750317e+00,
        -1.35984487e+00, -4.30918389e-02,  4.800761

In [12]:
np.array([np.sum(weights, axis=1),
          list(labels.values())])

array([[25.861123696719716, 0.203043416723395, 0.1655152802112303,
        0.5991356136936943, 0.2413446185999164, 0.6406430138787367,
        23.198200574360683, 0.9817688508695817, 1.4643263594209617,
        0.4344191223617231, 32.32226138326908, 1.3776481706985848,
        0.7702141795723927, 2.572388896416791, 1.3727265936473012,
        1.8547544570597458, 23.37406707865432, 25.687177938062625,
        224.59858133090498, 1.9525976681953603, 0.4998313558518762,
        1.8172984946047004, 0.8825259120714487, 3.044093334309192,
        1.9193177921159355, 4.139751109569765, 23.607907950367984],
       [None, 'bottle1', 'bottle1', 'bottle1', 'bottle1', 'bottle1',
        None, 'bottle1', 'bottle1', 'bottle1', 'buzzer', 'bottle1',
        'bottle1', 'bottle1', 'bottle1', 'bottle1', None, None, 'phone4',
        'bottle1', 'bottle1', 'bottle1', 'bottle1', 'bottle1', 'bottle1',
        'bottle1', None]], dtype=object)

In [13]:
labels = ['phone4', 'buzzer', 'bottle1']
columns_labels = {
    label: [c for c in res.columns if c.startswith(label)]
    for label in labels
  }


sum_spikes_df = pd.DataFrame()
sum_spikes_df['label'] = res['label']
for label, columns in columns_labels.items():
  sum_spikes_df[f'{label}'] = res[columns].sum(axis=1)
sum_spikes_df['predicted_label'] = sum_spikes_df[labels].idxmax(axis=1)
sum_spikes_df['success'] = sum_spikes_df['predicted_label'] == sum_spikes_df['label']
sum_spikes_df

,label,phone4,buzzer,bottle1,predicted_label,success
0,buzzer,1416.0,84.0,27.0,phone4,False
1,phone4,4709.0,72.0,30.0,phone4,True
2,bottle1,339.0,15.0,9.0,phone4,False
3,phone4,6072.0,55.0,20.0,phone4,True
4,buzzer,2985.0,46.0,27.0,phone4,False
5,buzzer,2962.0,50.0,21.0,phone4,False
6,buzzer,2252.0,64.0,24.0,phone4,False
7,bottle1,347.0,114.0,34.0,phone4,False
8,phone4,5446.0,12.0,14.0,phone4,True
9,bottle1,354.0,10.0,4.0,phone4,False
